In [114]:
import Pkg
Pkg.add("TimeSeries")

    Updating registry at `C:\Users\matsz\.julia\registries\General.toml`
   Resolving package versions...
   Installed MutableArithmetics ──────── v1.4.4
   Installed Libmount_jll ────────────── v2.40.1+0
   Installed MvNormalCDF ─────────────── v0.3.2
   Installed JpegTurbo_jll ───────────── v3.0.3+0
   Installed GR_jll ──────────────────── v0.73.5+0
   Installed OffsetArrays ────────────── v1.14.0
   Installed Unitful ─────────────────── v1.19.1
   Installed OpenSSL ─────────────────── v1.4.3
   Installed Polyester ───────────────── v0.7.14
   Installed Contour ─────────────────── v0.6.3
   Installed RecursiveArrayTools ─────── v3.18.1
   Installed BSplineKit ──────────────── v0.17.2
   Installed HTTP ────────────────────── v1.10.8
   Installed Cairo_jll ───────────────── v1.18.0+2
   Installed ColorSchemes ────────────── v3.25.0
   Installed Fontconfig_jll ──────────── v2.13.96+0
   Installed Libgpg_error_jll ────────── v1.49.0+0
   Installed Xorg_libSM_jll ──────────── v1.2.4+0
   

In [175]:
using DataFrames, FredData, XLSX, SHA, Dates, CSV, TimeSeries, StatsBase, HiddenMarkovModels, Distributions

In [116]:
usdpln = CSV.read("usdpln_m.csv", DataFrame)[:,[:Data,:Zamkniecie]]
usdpln = rename(usdpln,[:date, :usdpln])
usdpln.date = Date.(usdpln.date,dateformat"u dd, yyyy") 
usdpln

Row,date,usdpln
,Date,Float64
1,2000-12-29,4.128
2,2001-01-31,4.08
3,2001-02-28,4.0375
4,2001-03-30,4.0903
5,2001-04-30,3.9618
6,2001-05-31,3.9824
7,2001-06-29,4.0069
8,2001-07-31,4.2558
9,2001-08-31,4.239


In [117]:
acwi = DataFrame(XLSX.readdata("acwi_net_15.05.24.xlsx","History Index!A8:B288"), [:date,:acwi])
acwi.date = Date.(acwi.date,dateformat"u dd, yyyy") 
acwi.acwi = parse.(Float64,string.(acwi.acwi))
acwi

Row,date,acwi
,Date,Float64
1,2000-12-29,100.0
2,2001-01-31,102.514
3,2001-02-28,93.869
4,2001-03-30,87.516
5,2001-04-30,93.853
6,2001-05-31,92.75
7,2001-06-29,89.882
8,2001-07-31,88.447
9,2001-08-31,84.352


In [168]:
data = leftjoin(acwi,usdpln, on= :date)
data.acwi_pln = data.acwi .* data.usdpln


281-element Vector{Float64}:
  412.8
  418.25712
  378.9960875
  357.9666948
  371.8268154
  369.36760000000004
  360.1481858
  376.4127426
  357.568128
  323.9934875
    ⋮
 1503.37613752
 1526.71381512
 1428.2824351999998
 1480.1930220000002
 1526.90645538
 1562.9809198500002
 1625.9419341
 1675.18580625
 1649.7202854999998

In [169]:
returns = log.(data.acwi_pln[2:end]./data.acwi_pln[1:end-1])
returns = DataFrame([data.date[2:end],returns],[:date,:acwipln_returns,])


Row,date,acwipln_returns
,Date,Float64
1,2001-01-31,0.0131331
2,2001-02-28,-0.0985705
3,2001-03-30,-0.0570859
4,2001-04-30,0.0379882
5,2001-05-31,-0.00663584
6,2001-06-29,-0.0252768
7,2001-07-31,0.0441707
8,2001-08-31,-0.0513603
9,2001-09-28,-0.0986025


In [172]:
describe(returns[:,:acwipln_returns])

Summary Stats:
Length:         280
Missing Count:  0
Mean:           0.004948
Std. Deviation: 0.037267
Minimum:        -0.111985
1st Quartile:   -0.015893
Median:         0.007448
3rd Quartile:   0.029379
Maximum:        0.105947
Type:           Float64


In [178]:
init_guess = [0.5, 0.5]
trans_guess = [0.9 0.1; 0.1 0.9]
dists_guess = [Normal(0.014,  0.0231), Normal(-0.0037, 0.0563)]
hmm_guess = HMM(init_guess, trans_guess, dists_guess); 

hmm_est, loglikelihood_evolution = baum_welch(hmm_guess, returns[:,:acwipln_returns]);

In [179]:
last(loglikelihood_evolution)

545.5773581251411

In [180]:
transition_matrix(hmm_est)

2×2 Matrix{Float64}:
 0.959017  0.0409833
 0.115136  0.884864

In [182]:
dist = obs_distributions(hmm_est)

2-element Vector{Normal{Float64}}:
 Normal{Float64}(μ=0.011820057498208112, σ=0.026336061501808783)
 Normal{Float64}(μ=-0.012464488311656943, σ=0.052057226688356446)

In [185]:
println("regime 1")
println(mean(obs_distributions(hmm_est)[1])*12)
println(std(obs_distributions(hmm_est)[1])* 12^0.5)
println("regime 2")
println(mean(obs_distributions(hmm_est)[2])*12)
println(std(obs_distributions(hmm_est)[2])* 12^0.5)

regime 1
0.14184068997849736
0.09123079318478304
regime 2
-0.1495738597398833
0.18033152305072778


In [190]:
initialization(hmm_est)

2-element Vector{Float64}:
 5.243120667402711e-18
 1.0